<a href="https://colab.research.google.com/github/ramonVDAKKER/teaching/blob/main/actuarial_institute_crash_course_some_snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The basics of decision trees

This notebook contains snippets to facilitate a demo and discussion during a lecture. We focus on the technique and not on understanding the empirical application associated to the dataset we use.

## 0. Import packages

In [ ]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.inspection import plot_partial_dependence
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension
import pandas_profiling

## 1. Load dataset

In [ ]:
data = load_breast_cancer()
y_df = pd.DataFrame(data.target, columns=["target"])
X_df = pd.DataFrame(data.data, columns=data.feature_names)
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df, y_df, test_size=0.3, random_state=123)
data_train_df = pd.concat([y_train_df, X_train_df], axis=1)

In [ ]:
print(f"The dataset, available for training, has {len(data_train_df)} rows and {data_train_df.shape[1]} columns.")
print("Some rows of the dataset:")
display(data_train_df.head(5))

## 2. Data exploration

In [ ]:
profile = pandas_profiling.ProfileReport(data_train_df, title="Data report", html={"style": {"full_width": True}}, minimal=True)
profile

## 3. Splitting mechanism illustrated

In [ ]:
def gini_impurity(y_left, y_right):
  p_L = np.mean(y_left.values)
  gini_left = 2 * p_L * (1 - p_L)
  p_R = np.mean(y_right.values)
  gini_right = 2 * p_R * (1 - p_R)
  n_right = len(y_right)
  weight_right = n_right / (len(y_left) + n_right)
  gini = weight_right * gini_right + (1 - weight_right) * gini_left
  return gini, gini_left, gini_right
def determine_gini_impurity_for_feature_and_split_ext(y, X_df, name_feature, threshold):
  left = (X_df[name_feature] <= threshold)
  y_left = y[left]
  y_right = y[~ left]
  return gini_impurity(y_left, y_right) 
def determine_gini_impurity_for_feature_and_split(y, X_df, name_feature, threshold):
  gini, _, _ = determine_gini_impurity_for_feature_and_split_ext(y, X_df, name_feature, threshold)
  return gini
vector_determine_gini_impurity_for_feature_and_split = np.vectorize(determine_gini_impurity_for_feature_and_split, excluded=["y", "X_df"])
def analyze_feature(X_df, name_feature, y):
  feature_values = X_df[name_feature].sort_values(ascending=True)[1:-1]
  gini = vector_determine_gini_impurity_for_feature_and_split(y=y, X_df=X_df, name_feature=name_feature, threshold=feature_values)
  plt.plot(feature_values, gini)
  plt.ylabel("Gini")
  plt.xlabel("Splitvalue")
  plt.title(f"Gini as function of possible split values for feature {name_feature}.")
  print(f"Split value: {feature_values.iloc[gini.argmin()]}")
  print(f"Gini: {gini.min()}")

Let us analyze a feature and evaluate how we should split for this feature:

In [ ]:
name_feature = "mean perimeter"
sns.histplot(data=data_train_df, x=name_feature, hue="target", multiple="stack")

The following graph shows how the Gini varies with the threshold:

In [ ]:
analyze_feature(X_train_df, name_feature, y_train_df)

In [ ]:
def determine_best_split(y, X_df):
  out_df = pd.DataFrame(columns=["Gini", "Split"])
  for name_feature in X_df.columns:
    feature_values = X_df[name_feature].sort_values(ascending=True)[1:-1]
    gini = vector_determine_gini_impurity_for_feature_and_split(y=y, X_df=X_df, name_feature=name_feature, threshold=feature_values)
    out_df.loc[name_feature, "Split"] = feature_values.iloc[gini.argmin()]
    out_df.loc[name_feature, "Gini"] = gini.min()
  return out_df
display(determine_best_split(y_train_df, X_train_df).sort_values(by="Gini"))

## 4. Train decision tree using Scikit

Let us check that Scikit finds the same answer for the first split. To this end we train a decision tree with 1 as maximum depth.

In [ ]:
dtree = tree.DecisionTreeClassifier(max_depth=1)
dtree = dtree.fit(X_train_df, y_train_df)
tree.plot_tree(dtree) 

In [ ]:
name_feature = X_train_df.columns[20]
sns.histplot(data=data_train_df, x=name_feature, hue="target", multiple="stack")

Let us check if our ad hoc code yields the same Gini for both leaves:

In [ ]:
print(f"Gini before splitting:  {(2 * np.mean(y_train_df.values) * (1 - np.mean(y_train_df.values))):.03f}")
split_value = 16.795
gini, gini_left, gini_right = determine_gini_impurity_for_feature_and_split_ext(y_train_df, X_train_df, name_feature, split_value)
print(f"Gini left: {gini_left:.03f}.")
print(f"Gini right: {gini_right:.03f}.")
frac_left = np.mean(X_train_df[name_feature] <= split_value)
print(f"Gini after splitting: {frac_left * gini_left + (1 - frac_left) * gini_right:.03f}")

Now we understand how the decision tree comes to its splits, we train a "full" tree.

In [ ]:
dtree = tree.DecisionTreeClassifier()
dtree = dtree.fit(X_train_df, y_train_df)
tree.plot_tree(dtree) 

Use the tree to predict $Y$ for an observation $X$:

In [ ]:
x = X_train_df.loc[0]
print("For\n\n")
print(x)
print(f"\n\nthe tree yields Y_hat={dtree.predict(x.values.reshape(1, -1))[0]}.")
print(f"The true outcome is {y_train_df.loc[0][0]}.")

Now we "score" full dataset:

In [ ]:
y_train_pred = dtree.predict(X_train_df)

In [ ]:
print(y_train_pred)

Let us determine the confusion matrix:

In [ ]:
class_names = [0, 1]
def confusion_matrix(model_object, X_df, y_df):
    disp = plot_confusion_matrix(model_object, X_df, y_df)
    disp.ax_.set_title("Confusion matrix")
    plt.show()
    y_hat = model_object.predict(X_df)
    accuracy = accuracy_score(y_df, y_hat, normalize=True)
    print(f"The accuracy is {100 * accuracy:0.2f}%.")
confusion_matrix(dtree, X_train_df, y_train_df)

Next we evaluate the model on the test data:

In [ ]:
confusion_matrix(dtree, X_test_df, y_test_df)

Let us also fit a decision tree with restricted depth and evaluate its performance on the train and test set:

In [ ]:
dtree = tree.DecisionTreeClassifier(max_depth=3)
dtree = dtree.fit(X_train_df, y_train_df)
confusion_matrix(dtree, X_train_df, y_train_df)
confusion_matrix(dtree, X_test_df, y_test_df)

## 5. Random Forest using Scitkit

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train_df, y_train_df.values.ravel())

The random forest objects contains 100 trees. Let us have a look and calculate predictions, for a given observation, for all trees:

In [ ]:
x = X_train_df.loc[0].values.reshape(1, -1)
for t in range(0, 100):
  print(f"Prediction of tree {t}: Y_hat={ rf.estimators_[t].predict(x)[0]}")

You typically do not use this information, but rely directly on the predict-method:

In [ ]:
rf.predict(x)

Let us evaluate the performance of the random forest on the train and test set:

In [ ]:
confusion_matrix(rf, X_train_df, y_train_df)
confusion_matrix(rf, X_test_df, y_test_df)

As we have 100 decision trees, it is difficult to say how the features impact the predictions. Two standard tools are global feature importances and partial dependence plots.

In [ ]:
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
forest_importances = pd.Series(importances, index=X_train_df.columns)
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
plot_partial_dependence(rf, X_train_df, features = ["worst radius", "worst area"]) 

## 6. Fitting a neural network

In [ ]:
# initialize NN
nn = Sequential()
# add first layer with xxx nodes and using ReLu as activation function
nn.add(Dense(10, activation="relu", input_shape=(X_train_df.shape[1],), kernel_initializer="uniform"))
# add second layer with xxx nodes and using ReLu as activation function
nn.add(Dense(10, activation="relu", kernel_initializer="uniform"))
# add output layer (binary target)
nn.add(Dense(1, activation="sigmoid")) 

In [ ]:
# Compile the model
nn.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
# Verify that model contains information from compiling
print(f"Loss function: {nn.loss}")

In [ ]:
# Fit the model
nn.fit(X_train_df, y_train_df,  epochs=200, validation_split=0.3)

In [ ]:
y_hat_train_nn = nn.predict(X_train_df)
print(y_hat_train_nn[0:5])

In [ ]:
y_hat_train_nn = (y_hat_train_nn > .5)
accuracy = accuracy_score(y_train_df, y_hat_train_nn, normalize=True)
print(f"The accuracy is {100 * accuracy:0.2f}%.")


In [ ]:
y_hat_test_nn = nn.predict(X_test_df)
y_hat_test_nn = (y_hat_test_nn > .5)
accuracy = accuracy_score(y_test_df, y_hat_test_nn, normalize=True)
print(f"The accuracy is {100 * accuracy:0.2f}%.")
